In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
import nltk
import requests
import random
import math
import pathlib 
import modules.hein_scraping_functions

from modules.create_path import create_path
from modules.hein_scraping_functions import create_browser, webpage_wait, get_paper_data, mod_names, check_google, similar_names, search_names

In [2]:
# Create the paths for the data directories
input_path, work_path, intr_path, out_path, selenium_driver_path = create_path()

# Create the paths for the Chrome binary and selenium driver
chrome_binary_path = pathlib.Path("C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe")
selenium_driver_full_path = selenium_driver_path / "chromedriver.exe"

# Initalize the browsers that we are going to use
driver = create_browser(chrome_binary_path, selenium_driver_full_path)
g_driver = create_browser(chrome_binary_path, selenium_driver_full_path)

driver.get("http://proxy.its.virginia.edu/login?url=http://heinonline.org/HOL/Welcome")


In [3]:
# Data type: This is the type of data that we are using.
# The control group members only have one university url because they
# did not move. The lateral group members have two urls.
data_type = "control"

# Load the dataset from the working directory
# The dataset in the working directory has already 
# been cleaned.
data = pd.read_excel(work_path / "{}.xlsx".format(data_type))

# Load the name modification dataset
name_mod = pd.read_excel(input_path / "name_mod_{}.xlsx".format(data_type))

# MERGE: Right join the name mod data to get the observations with a different last name
alt_name_diff_last_name = pd.merge(data, name_mod[["ID", "diff_last_name"]], how = "right", left_on = ["ID"], right_on = ["ID"])

# Subset to the non-missing different last names
alt_name_diff_last_name = alt_name_diff_last_name[alt_name_diff_last_name["diff_last_name"].notnull()]
# Create a copy of the alternate last name observations with a different last name
alt_name_diff_last_name["LastName"] = alt_name_diff_last_name["diff_last_name"]
# Drop the diff_last_name column
alt_name_diff_last_name = alt_name_diff_last_name.drop(["diff_last_name"], axis = 1)

# Add the observations onto the main dataset
data = pd.concat([data, alt_name_diff_last_name], ignore_index = True)

# Flag any observations that now show up multiple times
data["multi_obs"] = data["ID"].duplicated(keep=False)

data

,Short URL Destination,Short URL Origin,FirstName,LastName,ID,Lateral,LateralYear,Origin School,Destination School,multi_obs
0,duke.edu,upenn.edu,Matthew,Adler,1,1,2012,University of Pennsylvania Law School,Duke University School of Law,False
1,duke.edu,northwestern.edu,Michael,Frakes,80,1,2016,Northwestern University Pritzker School of Law,Duke University School of Law,False
2,duke.edu,virginia.edu,Brandon,Garrett,88,1,2018,University of Virginia School of Law,Duke University School of Law,False
3,duke.edu,gwu.edu,H.,Powell,216,1,2012,George Washington University Law School,Duke University School of Law,False
4,duke.edu,uc.edu,Darrell,Miller,182,1,2013,University of Cincinnati College of Law,Duke University School of Law,False
...,...,...,...,...,...,...,...,...,...,...
294,northeastern.edu,wlu.edu,Victoria,Shannon,235,1,2017,Washington and Lee University School of Law,Arizona State Sandra Day O'Connor College of Law,True
295,rutgers.edu,ucdavis.edu,Rose,Cuison,273,1,2017,UC Davis School of Law,Rutgers Law Newark,True
296,buffalo.edu,pace.edu,Luis,Aponte,47,1,2013,Pace University Elisabeth Haub School of Law,University at Buffalo School of Law,True
297,gwu.edu,wfu.edu,Emily,Meazell,103,1,2014,Wake Forest University School of Law,George Washington University Law School,True


In [4]:
# Create a list to store the alternate name data
alt_names_data = []
# Create the alt-names dataset column list
alt_names_columns = ["ID", "FirstName", "LastName", "fm_names", "err_fm_names", "multi_obs"]

# Check to see if the file for the alternate names data already exists.
# If it does, we only want to looks for the missing observations
alt_names_file = intr_path / "alt_names.xlsx"
if alt_names_file.exists():
    print("Data already exists. Names that have already been scraped will be skipped")
    # Set the append flag to 1
    append = 1
    # Create the dataset of existing alt names.
    df_existing_alt_names = pd.read_excel(alt_names_file)
    # Complete a left outer join of the existing alt names and the lateral/control data to get 
    # a list of the names that we still need to scrape alt names for.
    data = pd.merge(data, df_existing_alt_names["ID"], how = "outer", left_on = "ID", right_on = "ID", indicator=True)
    data = data[data['_merge'] == 'left_only']
    data = data.drop(["_merge"], axis = 1)
else:
    # Set the append flag to zero because we won't have any data to append
    append = 0
# Data contains the observations that were not available in the existing alt_names file.
# If there is no existing file, it just contains the lateral/control data.
data.reset_index(inplace = True, drop = True)


Data already exists. Names that have already been scraped will be skipped


In [5]:
data

,Short URL Destination,Short URL Origin,FirstName,LastName,ID,Lateral,LateralYear,Origin School,Destination School,multi_obs
0,fsu.edu,ubc.ca,Shi-Ling,Hsu,123,1,2012,University of British Columbia,Florida State University College of Law,True
1,fsu.edu,ubc.ca,Shi-Ling,"Hsu,",123,1,2012,University of British Columbia,Florida State University College of Law,True


In [6]:
#This loop goes through each name
for i in range(len(data)):
    #This section gets the professor's information from the dataframe 
    # Get variable values from the dataframe
    prof_id = data['ID'][i]
    mid_first_name = data['FirstName'][i]
    last_name = data['LastName'][i]
    full_name = mid_first_name + ' ' +  last_name
    #This line gets the school URLs from the dataframe
    if data_type == "lateral":
        school_url = [data['Short URL Origin'][i], data['Short URL Destination'][i]]
        school = data['Origin School'][i]
        new_school = data['Destination School'][i]
    elif data_type == "control":
        school_url = [data['Short URL Origin'][i]]
        school = data['Origin School'][i]
    multi_obs = data["multi_obs"][i]

    # Print the name that we are considering
    print(full_name)
    
    # Search by author to find potential alternative first and middle names:
    print(school_url)
    fm_names, err_fm_names = search_names(mid_first_name, last_name, school_url, driver, g_driver)


    # Create a list of values to append to the dataframe
    # We convert fm_names and err_fm_names to lists of strings during this step
    values_alt_names = [prof_id, mid_first_name, last_name, fm_names, err_fm_names, multi_obs]
    dict_values_alt_names = dict(zip(alt_names_columns, values_alt_names))
    alt_names_data.append(dict_values_alt_names)

Shi-Ling Hsu
['ubc.ca', 'fsu.edu']
URL: ubc.ca
ubc.ca in: https://commons.allard.ubc.ca/cgi/viewcontent.cgi?article=1022&context=emeritus_pubs
URL: ubc.ca
ubc.ca in: https://commons.allard.ubc.ca/cgi/viewcontent.cgi?article=1022&context=emeritus_pubs
Shi-Ling Hsu,
['ubc.ca', 'fsu.edu']


In [7]:
# Add the names back into the dataframe of alternate names
df_alt_names = pd.DataFrame(columns = alt_names_columns)

# Only complete these steps if we have data to add
if alt_names_data:
    # Append all of the data to the ouput dataset and output to Excel
    df_alt_names = df_alt_names.append(alt_names_data)

    # MERGE: Merge on the other variables from the lateral data
    alt_name_full = pd.merge(df_alt_names, lateral.drop(["FirstName", "LastName"], axis = 1), how = "left", left_on = "ID", right_on = "ID")

else:
    alt_name_full = df_alt_names_final

# # If there was already output data, we append the data to the existing data
if append == 1:
    alt_names = pd.concat([df_existing_alt_names, alt_name_full], ignore_index = True)
else:
    alt_names = alt_name_full

# DROP: Drop extra columns
# alt_names.drop(["diff_last_name", "mid_first_name", "last_name"], axis = 1, inplace = True)

# Export to Excel
alt_names.to_excel(intr_path / "alt_names.xlsx", index = False)

In [8]:
alt_names

,Unnamed: 0,ID,Short URL Destination,Short URL Origin,FirstName,LastName,Lateral,LateralYear,Origin School,Destination School,fm_names,err_fm_names,multi_obs
0,0.0,1,duke.edu,upenn.edu,Matthew,Adler,1,2012,University of Pennsylvania Law School,Duke University School of Law,"Matthew, Matthew D.",Matthew H.,False
1,1.0,2,gsu.edu,avemarialaw.edu,Edward,Afield,1,2016,Ave Maria School of Law,Georgia State College of Law,W. Edward,NaN,False
2,2.0,3,utexas.edu,bc.edu,Richard,Albert,1,2017,Boston College Law School,University of Texas School of Law,Richard,NaN,False
3,3.0,4,tamu.edu,wisc.edu,Lisa,Alexander,1,2016,University of Wisconsin Law School,Texas A&M University School of Law,Lisa T.,Lisa E.,False
4,4.0,5,american.edu,suffolk.edu,Hilary,Allen,1,2018,Suffolk University Law School,American University Washington College of Law,Hilary J.,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,NaN,47,buffalo.edu,pace.edu,Luis,Aponte,1,2013,Pace University Elisabeth Haub School of Law,University at Buffalo School of Law,"Luis Ernesto Chiesa, Luis E. Chiesa",NaN,True
295,NaN,103,gwu.edu,wfu.edu,Emily,Hammond,1,2014,Wake Forest University School of Law,George Washington University Law School,Emily,NaN,True
296,NaN,103,gwu.edu,wfu.edu,Emily,Meazell,1,2014,Wake Forest University School of Law,George Washington University Law School,Emily Hammond,NaN,True
297,NaN,123,fsu.edu,ubc.ca,Shi-Ling,Hsu,1,2012,University of British Columbia,Florida State University College of Law,"Shi-Ling, Shi-ling",,True
